점수 : 13.64963  등수 :418

In [1]:
import pandas as pd
import numpy as np
import os
import random
from xgboost import XGBRegressor
import warnings

warnings.filterwarnings(action='ignore')

def seed_everything(seed):
    """
    모든 난수 생성기의 시드를 고정하여 재현성을 보장하는 함수입니다.
    """
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

# 시드 고정
seed_everything(42)

# 1. 데이터 불러오기
# train.csv, test.csv, building_info.csv 파일을 불러옵니다.
train_df = pd.read_csv('dataset/train.csv', encoding='utf-8')
test_df = pd.read_csv('dataset/test.csv', encoding='utf-8')
building_info = pd.read_csv('dataset/building_info.csv', encoding='utf-8')


building_info = building_info.replace('-', '0') # 숫자로 변환하기 위해 문자 '0'으로 변경

# '태양광용량', 'ESS저장용량', 'PCS용량' 컬럼을 숫자형(float)으로 변환
building_info['태양광용량(kW)'] = building_info['태양광용량(kW)'].astype(float)
building_info['ESS저장용량(kWh)'] = building_info['ESS저장용량(kWh)'].astype(float)
building_info['PCS용량(kW)'] = building_info['PCS용량(kW)'].astype(float)

# '건물유형'을 원-핫 인코딩
building_type_dummies = pd.get_dummies(building_info['건물유형'], prefix='건물유형')
building_info = pd.concat([building_info, building_type_dummies], axis=1)
building_info.drop('건물유형', axis=1, inplace=True)


# 3. 데이터 병합
# train, test 데이터에 건물 정보를 결합.
train_df = pd.merge(train_df, building_info, on='건물번호', how='left')
test_df = pd.merge(test_df, building_info, on='건물번호', how='left')


# 4. 피처 엔지니어링 (Feature Engineering)
# '일시' 컬럼을 datetime 형식으로 변환하여 시간 관련 특징을 쉽게 추출할 수 있도록 한다.
train_df['일시'] = pd.to_datetime(train_df['일시'], format='%Y%m%d %H')
test_df['일시'] = pd.to_datetime(test_df['일시'], format='%Y%m%d %H')

# 시간 관련 파생변수 생성
for df in [train_df, test_df]:
    df['month'] = df['일시'].dt.month
    df['day'] = df['일시'].dt.day
    df['hour'] = df['일시'].dt.hour
    df['dayofweek'] = df['일시'].dt.dayofweek # 요일 (0:월요일, 6:일요일)

    # 시간의 주기성을 나타내기 위한 sin, cos 변환
    # 23시와 0시는 시간상으로 1시간 차이밖에 나지 않지만, 숫자로는 23만큼의 큰 차이가 난다.
    # sin, cos 변환을 통해 이러한 주기적 특성을 모델에 알려줍니다.
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['dayofweek_sin'] = np.sin(2 * np.pi * df['dayofweek'] / 7)
    df['dayofweek_cos'] = np.cos(2 * np.pi * df['dayofweek'] / 7)

    # === 여기부터 변경: 체감온도(WCI, Wind Chill Index) ===
    # T: 기온(°C), V: 풍속(km/h)  → 보통 데이터는 m/s이므로 km/h로 변환(×3.6)
    # 유효범위: T <= 10°C 그리고 V >= 4.8 km/h, 아니면 관례적으로 T를 반환
    wind_col = '풍속(m/s)' if '풍속(m/s)' in df.columns else ('풍속' if '풍속' in df.columns else None)
    if wind_col is not None:
        v_kmh = df[wind_col] * 3.6  # 만약 이미 km/h라면 곱하기 3.6 제거
        v_pow = np.power(v_kmh, 0.16)
        wci_raw = 13.12 + 0.6215 * df['기온(°C)'] - 11.37 * v_pow + 0.3965 * df['기온(°C)'] * v_pow
        cond = (df['기온(°C)'] <= 10) & (v_kmh >= 4.8)
        df['WCI'] = np.where(cond, wci_raw, df['기온(°C)'])
    else:
        # 풍속 컬럼이 없으면 안전하게 실제 기온을 사용
        df['WCI'] = df['기온(°C)']

# 5. 학습 데이터 준비
drop_cols_train = ['num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)', 'THI']  # THI는 사용 안 함
train_x = train_df.drop(columns=drop_cols_train, errors='ignore')
train_y = train_df['전력소비량(kWh)']

drop_cols_test = ['num_date_time', '일시', 'THI']  # 동일하게 THI 제외 (혹시 존재할 경우)
test_x = test_df.drop(columns=drop_cols_test, errors='ignore')

# 컬럼 정렬/동기화 (안전하게 reindex 사용)
test_x = test_x.reindex(columns=train_x.columns, fill_value=0)

# train_x와 test_x의 컬럼 순서를 동일하게 맞춤
test_x = test_x[train_x.columns]

# 6. 모델 학습 및 예측
model = XGBRegressor(seed=42)
model.fit(train_x, train_y)

preds = model.predict(test_x)

# 7. 제출 파일 생성
submission = pd.read_csv('dataset/sample_submission.csv')
submission['answer'] = preds
submission.to_csv('submission.csv', index=False)

print("모델 학습 및 예측이 완료되었습니다. submission.csv 파일이 생성되었습니다.")

모델 학습 및 예측이 완료되었습니다. submission.csv 파일이 생성되었습니다.
